In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt

from modules.utils import load_cifar10, load_cifar100
from modules.cnn_with_spectral_pooling import CNN_Spectral_Pool

% matplotlib inline
% load_ext autoreload
% autoreload 2

# INSTRUCTIONS - PLEASE READ CAREFULLY

### GPU Required

The code will only run on a machine with a GPU since it processes images channels-first.

### 32 GB RAM Required

Unlike the CIFAR-10 dataset, the CIFAR-100 dataset does not come in batches. Therefore, loading the dataset requires at least 32 GB of RAM.

### Unzipping model Required

You need to run 

```tar -zxvf best_model_100.tar.gz```

from the src/ directory before proceeding.

In [ ]:
print('Please make sure you are running on a machine with a GPU, at least 32 GB RAM, and have already unzipped the best_model_100.tar.gz file before proceeding.')

# Download CIFAR data, if necessary, and load it

In [2]:
xtrain, ytrain, xtest, ytest = load_cifar100(get_test_data=True, channels_last=False)

file already downloaded..


In [3]:
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

((50000, 3, 32, 32), (50000,), (10000, 3, 32, 32), (10000,))

# Best result from manual tuning

In [8]:
# Manual tuning
tf.reset_default_graph()
cnn = CNN_Spectral_Pool(M=4,
                        verbose=True,
                        num_output=100,
                        gamma=0.79)

cnn.calc_test_accuracy(xtest, ytest, 'best_model_cifar_100/test_1513439725.3974495')

Building tf graph...
Adding conv layer 1 | Input size: 32 | Input channels: 3 | #filters: 128 | filter size: 3
Adding spectral pool layer 1 | Input size: 32 | filter size: (25,25) | Freq Dropout Bounds: (3.4125,13.0)
Adding conv layer 2 | Input size: 25 | Input channels: 128 | #filters: 160 | filter size: 3
Adding spectral pool layer 2 | Input size: 25 | filter size: (19,19) | Freq Dropout Bounds: (2.25,10.0)
Adding conv layer 3 | Input size: 19 | Input channels: 160 | #filters: 192 | filter size: 3
Adding spectral pool layer 3 | Input size: 19 | filter size: (15,15) | Freq Dropout Bounds: (1.5,8.0)
Adding conv layer 4 | Input size: 15 | Input channels: 192 | #filters: 224 | filter size: 3
Adding spectral pool layer 4 | Input size: 15 | filter size: (11,11) | Freq Dropout Bounds: (0.8999999999999999,6.0)
Adding conv layer 5 | Input size: 11 | Input channels: 224 | #filters: 224 | filter size: 1
Adding conv layer 6 | Input size: 11 | Input channels: 224 | #filters: 100 | filter size: 1


# Build graph with optimal hyperparameters

In [4]:
from modules.cnn_with_spectral_pooling import CNN_Spectral_Pool
validation_size = 4096

cnn = CNN_Spectral_Pool(M=7,
                        num_output=100,
                        verbose=True,
                        learning_rate=1.09e-3,
                        l2_norm=2.7e-5,
                        lr_reduction_factor=0.5,
                        lr_reduction_epochs=[11,21,31,41],
                        gamma=0.6336)


In [5]:
tf.reset_default_graph()
cnn.train(xtrain[:-validation_size],
          ytrain[:-validation_size],
          xtrain[-validation_size:],
          ytrain[-validation_size:],
          batch_size=256,
          epochs=50,
          extra_conv_layer=True,
          use_global_averaging=True,
          model_name='optimal_hyperparams'
)
full_model_name = cnn.full_model_name

Building tf graph...
Adding conv layer 1 | Input size: 32 | Input channels: 3 | #filters: 128 | filter size: 3
Adding spectral pool layer 1 | Input size: 32 | filter size: (20,20) | Freq Dropout Bounds: (3.0642857142857145,11.0)
Adding conv layer 2 | Input size: 20 | Input channels: 128 | #filters: 160 | filter size: 3
Adding spectral pool layer 2 | Input size: 20 | filter size: (12,12) | Freq Dropout Bounds: (1.7999999999999998,7.0)
Adding conv layer 3 | Input size: 12 | Input channels: 160 | #filters: 192 | filter size: 3
Adding spectral pool layer 3 | Input size: 12 | filter size: (7,7) | Freq Dropout Bounds: (0.9428571428571428,4.0)
Adding conv layer 4 | Input size: 7 | Input channels: 192 | #filters: 224 | filter size: 3
Adding spectral pool layer 4 | Input size: 7 | filter size: (4,4) | Freq Dropout Bounds: (0.6428571428571429,3.0)
Adding conv layer 5 | Input size: 4 | Input channels: 224 | #filters: 256 | filter size: 3
Adding spectral pool layer 5 | Input size: 4 | filter size:

45824/45904 loss: 1.5056647062301636 | training accuracy: 57.922% | validation accuracy : 32.666%
training epoch 41 
	Learning rate reduced to 6.8125e-05 at epoch 41
45824/45904 loss: 1.5088940858840942 | training accuracy: 57.893% | validation accuracy : 32.690%
training epoch 42 
45824/45904 loss: 1.4472187757492065 | training accuracy: 59.565% | validation accuracy : 33.154%

	Best validation accuracy! iteration:7518 accuracy: 33.154296875%

training epoch 43 
45824/45904 loss: 1.3882346153259277 | training accuracy: 60.846% | validation accuracy : 32.739%
training epoch 44 
45824/45904 loss: 1.4145677089691162 | training accuracy: 60.278% | validation accuracy : 32.593%
training epoch 45 
45824/45904 loss: 1.4203919172286987 | training accuracy: 60.324% | validation accuracy : 32.812%
training epoch 46 
45824/45904 loss: 1.4002615213394165 | training accuracy: 60.488% | validation accuracy : 32.788%
training epoch 47 
45824/45904 loss: 1.3819750547409058 | training accuracy: 61.343

# Calculate accuracy on test set

In [6]:
tf.reset_default_graph()
cnn.calc_test_accuracy(xtest, ytest, 'optimal_hyperparams/{0}'.format(full_model_name))

Building tf graph...
Adding conv layer 1 | Input size: 32 | Input channels: 3 | #filters: 128 | filter size: 3
Adding spectral pool layer 1 | Input size: 32 | filter size: (20,20) | Freq Dropout Bounds: (3.0642857142857145,11.0)
Adding conv layer 2 | Input size: 20 | Input channels: 128 | #filters: 160 | filter size: 3
Adding spectral pool layer 2 | Input size: 20 | filter size: (12,12) | Freq Dropout Bounds: (1.7999999999999998,7.0)
Adding conv layer 3 | Input size: 12 | Input channels: 160 | #filters: 192 | filter size: 3
Adding spectral pool layer 3 | Input size: 12 | filter size: (7,7) | Freq Dropout Bounds: (0.9428571428571428,4.0)
Adding conv layer 4 | Input size: 7 | Input channels: 192 | #filters: 224 | filter size: 3
Adding spectral pool layer 4 | Input size: 7 | filter size: (4,4) | Freq Dropout Bounds: (0.6428571428571429,3.0)
Adding conv layer 5 | Input size: 4 | Input channels: 224 | #filters: 256 | filter size: 3
Adding spectral pool layer 5 | Input size: 4 | filter size: